In [ ]:
# base libraries
import numpy as np
import pandas as pd
import regex as re
import itertools
import sklearn.metrics as sk
from functools import reduce

# deep learning libraries
import torch
import transformers
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# hyperparameter optimization
import optuna
from optuna.samplers import TPESampler
import optuna.visualization.matplotlib as oviz

# file system manipulation
import os
import shutil
from pathlib import Path
import pickle
import copy

# logging
import logging
import time

In [ ]:
# set seeds to make computations deterministic
np.random.seed(42)
torch.manual_seed(42)

# check CUDA availability
cuda_available = torch.cuda.is_available()
print("Is CUDA available? ", "Yes" if cuda_available else "No")

In [ ]:
final_test = pd.read_csv(r"Storage/Bert/test_8_11.csv")
final_test.columns = ["PatientID", "text", "labels"]

In [ ]:
best_model = ClassificationModel(
    "bert",
    "Storage/Bert/NoHyperParameterTuningResults/trial_0"
)

In [ ]:
test_results, test_outputs, test_wrong = best_model.eval_model(
    final_test,
)

In [ ]:
max_prob_list = []
test_prob_list = []
test_pred_list = []
for i in range(len(final_test)):
    # prob_list = list(torch.softmax(torch.from_numpy(model_outputs[i]), axis=0)[:,1])
    prob_list = torch.softmax(torch.from_numpy(test_outputs[i]), axis=0)
    #print("Prob List: ", prob_list, type(prob_list))

    extracted_prob_list = []
    for i in range(len(prob_list)):
        extracted_prob_list.append(float(prob_list[i]))

    #print("Extracted Prob List: ", extracted_prob_list)
    # find max one in each submatrix of length 3
    max_proba = max(extracted_prob_list)

    # identify model prediction based on location of max_proba within extracted_prob_list
    if (extracted_prob_list[0] == max_proba):
        test_pred_list.append(0)
    elif (extracted_prob_list[1] == max_proba):
        test_pred_list.append(1)
    else:
        test_pred_list.append(2)

    max_prob_list.append(max_proba)
    test_prob_list.append(extracted_prob_list)

In [ ]:
final_test["pred"] = test_pred_list
final_test["proba"] = max_prob_list

In [ ]:
revised_preds = pd.read_csv(r"Storage/Bert/Revised Bert Preds.csv")

In [ ]:
edits_made = 0
for i in range(len(final_test)):
    for j in range(len(revised_preds)):
        if (final_test.at[i, "text"] == revised_preds.at[j, "text"]):
            edits_made += 1
            final_test.at[i, "labels"] = revised_preds.at[j, "labels"]

In [ ]:
sk.accuracy_score(test_pred_list, final_test["labels"])

In [ ]:
final_test.head()

In [ ]:
sk.roc_auc_score(final_test["labels"], test_prob_list, multi_class = "ovr", average = "weighted")

In [ ]:
import pickle

f = open("Storage/Bert/test_set_prob.pkl", "wb")
pickle.dump(str(test_prob_list), f)
f.close()

In [ ]:
from sklearn import metrics

target_names = ['Negative', 'Neither', 'Positive']
results = metrics.classification_report(final_test["labels"], final_test["pred"], target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = metrics.confusion_matrix(final_test["labels"], final_test["pred"])
print("Micro F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "micro"))
print("Macro F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "macro"))
print("Weighted F1: ", metrics.f1_score(final_test["labels"], final_test["pred"], average = "weighted"))
print("Confusion Matrix: \n", conf_mat)
print("Classification Report: \n", results)

In [ ]:
FP = conf_mat.sum(axis = 0) - np.diag(conf_mat) 
FN = conf_mat.sum(axis = 1) - np.diag(conf_mat)
TP = np.diag(conf_mat)
TN = conf_mat.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)
print("FPR: ", FPR)

In [ ]:
(0.93121693 +  0.96855346 + 0.98739496) / 3

In [ ]:
(0.98076923 + 0.93283582 + 0.81818182) / 3